In [24]:
from collections.abc import Collection, Hashable, Iterable, Iterator
from typing import Literal

import networkx as nx
import numpy as np
from networkx import Graph
from scipy.sparse import csr_matrix

from toponetx.classes.complex import Complex
from toponetx.classes.hyperedge import HyperEdge
from toponetx.classes.reportviews import HyperEdgeView, NodeView
from toponetx.classes.simplex import Simplex
from toponetx.classes.simplicial_complex import SimplicialComplex
from toponetx.exception import TopoNetXError
from toponetx.utils.structure import (
    incidence_to_adjacency,
    sparse_array_to_neighborhood_dict,
)

In [ ]:
from toponetx import CombinatorialComplex as cc
import pytest
def test_remove_nodes():
  example = cc(name = 'hello')
  example.add_cell([1, 2], rank=1)
  example.add_cell([1, 3,2], rank=1)
  example.add_cell([1, 2, 4, 3], rank=2)
  example.add_cell([2, 5], rank=1)
  example.add_cell([2, 6, 4], rank=2)
  example.remove_node(1)
  assert example._complex_set.hyperedge_dict == {1: {frozenset({2, 5}): {'weight': 1},
    frozenset({2}): {'weight': 1},
    frozenset({2, 3}): {'weight': 1}},
  0: {frozenset({2}): {'weight': 1},
    frozenset({3}): {'weight': 1},
    frozenset({4}): {'weight': 1},
    frozenset({5}): {'weight': 1},
    frozenset({6}): {'weight': 1}},
  2: {frozenset({2, 4, 6}): {'weight': 1}, frozenset({2, 3, 4}): {'weight': 1}}}
  with pytest.raises(KeyError):
      example.remove_nodes([1]) ==  KeyError(f"node 1 not in CombinatorialComplex")
  example.remove_nodes([2,5])
  assert example._complex_set.hyperedge_dict == {1: {frozenset({3}): {'weight': 1}},
  0: {frozenset({3}): {'weight': 1},
    frozenset({4}): {'weight': 1},
    frozenset({6}): {'weight': 1}},
  2: {frozenset({4, 6}): {'weight': 1}, frozenset({3, 4}): {'weight': 1}}}
  assert example._aux_complex.simplices.faces_dict == [{frozenset({3}): {'is_maximal': True, 'membership': set()},
    frozenset({4}): {'is_maximal': True, 'membership': set()},
    frozenset({6}): {'is_maximal': True, 'membership': set()}},
  {frozenset({3, 4}): {'is_maximal': True, 'membership': set()},
    frozenset({4, 6}): {'is_maximal': True, 'membership': set()}}]
  example.remove_nodes(HyperEdge([3]))
  assert example._complex_set.hyperedge_dict == {0: {frozenset({4}): {'weight': 1}, frozenset({6}): {'weight': 1}},
  2: {frozenset({4, 6}): {'weight': 1}, frozenset({4}): {'weight': 1}}}
  node = {4:3}
  with pytest.raises(TypeError):
      example.remove_node(node) ==  TypeError("node must be a HyperEdge or a hashable object")
  example = cc(name = 'hello')
  example.add_cell([1, 2], rank=1)
  example.add_cell([1, 3,2], rank=1)
  example.add_cell([1, 2, 4, 3], rank=2)
  example.add_cell([2, 5], rank=1)
  example.add_cell([2, 6, 4], rank=2)
  example.remove_nodes([HyperEdge([1]),HyperEdge([2])])
  assert example._complex_set.hyperedge_dict == {1: {frozenset({3}): {'weight': 1}, frozenset({5}): {'weight': 1}},
  0: {frozenset({3}): {'weight': 1},
    frozenset({4}): {'weight': 1},
    frozenset({5}): {'weight': 1},
    frozenset({6}): {'weight': 1}},
  2: {frozenset({3, 4}): {'weight': 1}, frozenset({4, 6}): {'weight': 1}}}

In [ ]:
from toponetx import CombinatorialComplex as cc
def test_set_cell_attributes(self):
    CC = cc(name = 'hello')
    CC.add_cell([1, 2], rank=1)
    CC.add_cell([1, 3,2], rank=1)
    CC.add_cell([1, 2, 3, 4], rank=2)
    CC.add_cell([2, 5], rank=1)
    CC.add_cell([2, 6, 4], rank=2)
    d = {(1, 2, 3, 4): 'red', (1, 2, 3): 'blue'}
    CC.set_cell_attributes(d, name='color')
    assert CC.cells[(1, 2, 3, 4)]['color'] == 'red'
    d = {(1, 2):{'attr1':'blue','size':'large'}}
    CC.set_cell_attributes(d)
    assert CC.cells[(1, 2)]['attr1'] == 'blue'

In [46]:
CC = cc()
CC.add_cell([1, 2, 3], rank=2)
import pytest
with pytest.raises(ValueError) as ex:
    CC.add_cell(HyperEdge([1]),3)
assert str(ex.value) == "violation of the combinatorial complex condition : the hyperedge frozenset({1, 2, 3}) in the complex has rank 2 is smaller than 3, the rank of the input hyperedge frozenset({1}) "

In [47]:
from toponetx import CombinatorialComplex as cc
example1 = cc(name = 'hello')
example1.add_cell([4,6], rank=2)
example1._add_nodes_from([1,2,3])
example1._complex_set.hyperedge_dict

{2: {frozenset({4, 6}): {'weight': 1}},
 0: {frozenset({4}): {'weight': 1}, frozenset({6}): {'weight': 1}}}

{2: {frozenset({4, 6}): {'weight': 1}},
 0: {frozenset({4}): {'weight': 1},
  frozenset({6}): {'weight': 1},
  frozenset({1}): {'weight': 1},
  frozenset({2}): {'weight': 1},
  frozenset({3}): {'weight': 1}}}

In [ ]:
# class MyCCComplex(Complex):
#     def __init__(
#         self,
#         cells: Collection | None = None,
#         name: str = "",
#         ranks: Collection | None = None,
#         graph_based: bool = False,
#         **kwargs,
#     ) -> None:
#         super().__init__(name, **kwargs)

#         self.graph_based = graph_based  
#         self._aux_complex = SimplicialComplex()

#         self._complex_set = HyperEdgeView()

#         if cells is not None:
#             if not isinstance(cells, Iterable):
#                 raise TypeError(
#                     f"Input cells must be given as Iterable, got {type(cells)}."
#                 )

#             if not isinstance(cells, Graph):
#                 if ranks is None:
#                     for cell in cells:
#                         if not isinstance(cell, HyperEdge):
#                             raise ValueError(
#                                 f"input must be an HyperEdge {cell} object when rank is None"
#                             )
#                         if cell.rank is None:
#                             raise ValueError(f"input HyperEdge {cell} has None rank")
#                         self.add_cell(cell, cell.rank)
#                 else:
#                     if isinstance(cells, Iterable) and isinstance(ranks, Iterable):
#                         if len(cells) != len(ranks):
#                             raise TopoNetXError(
#                                 "cells and ranks must have equal number of elements"
#                             )
#                         else:
#                             for cell, rank in zip(cells, ranks):
#                                 self.add_cell(cell, rank)
#                 if isinstance(cells, Iterable) and isinstance(ranks, int):
#                     for cell in cells:
#                         self.add_cell(cell, ranks)
#             else:
#                 for node in cells.nodes:  # cells is a networkx graph
#                     self.add_node(node, **cells.nodes[node])
#                 for edge in cells.edges:
#                     u, v = edge
#                     self.add_cell([u, v], 1, **cells.get_edge_data(u, v))
#     def __contains__(self, item) -> bool:
#         return item in self.nodes
#     def __getitem__(self, node):
#         return self.nodes[node]
#     def __str__(self) -> str:
#         return f"Combinatorial Complex with {len(self.nodes)} nodes and cells with ranks {self.ranks} and sizes {self.shape} "
#     def __repr__(self) -> str:
#         return f"CombinatorialComplex(name='{self.name}')"
#     def __len__(self) -> int:
#         return len(self.nodes)
#     def __iter__(self) -> Iterator:
#         return iter(self.nodes)
#     def _add_node(self, node, **attr) -> None:
#         self._add_hyperedge(hyperedge=node, rank=0, **attr)

#     def add_node(self, node, **attr) -> None:
#         self._add_node(node, **attr)

#     def adjacency_matrix(self, rank, via_rank, s: int = 1, index: bool = False):
#         if via_rank is not None:
#             assert rank < via_rank
#         if index:
#             B, row, col = self.incidence_matrix(
#                 rank, via_rank, sparse=True, index=index
#             )
#         else:
#             B = self.incidence_matrix(
#                 rank, via_rank, incidence_type="up", sparse=True, index=index
#             )
#         A = incidence_to_adjacency(B.T, s=s)
#         if index:
#             return A, row
#         return A
    
#     def _incidence_matrix_helper(
#     self, children, uidset, sparse: bool = True, index: bool = False
#     ):
#         """Help compute the incidence matrix."""
#         from collections import OrderedDict
#         from operator import itemgetter

#         ndict = dict(zip(children, range(len(children))))
#         edict = dict(zip(uidset, range(len(uidset))))

#         ndict = OrderedDict(sorted(ndict.items(), key=itemgetter(1)))
#         edict = OrderedDict(sorted(edict.items(), key=itemgetter(1)))

#         r_hyperedge_dict = {j: children[j] for j in range(len(children))}
#         k_hyperedge_dict = {i: uidset[i] for i in range(len(uidset))}

#         r_hyperedge_dict = OrderedDict(
#             sorted(r_hyperedge_dict.items(), key=itemgetter(0))
#         )
#         k_hyperedge_dict = OrderedDict(
#             sorted(k_hyperedge_dict.items(), key=itemgetter(0))
#         )

#         if len(ndict) != 0:

#             # if index:
#             #     rowdict = {v: k for k, v in ndict.items()}
#             #     coldict = {v: k for k, v in edict.items()}

#             if sparse:
#                 # Create csr sparse matrix
#                 rows = list()
#                 cols = list()
#                 data = list()
#                 for n in ndict:
#                     for e in edict:
#                         if n <= e:
#                             data.append(1)
#                             rows.append(ndict[n])
#                             cols.append(edict[e])
#                 MP = csr_matrix(
#                     (data, (rows, cols)),
#                     shape=(len(r_hyperedge_dict), len(k_hyperedge_dict)),
#                 )
#             else:
#                 # Create an np.matrix
#                 MP = np.zeros((len(children), len(uidset)), dtype=int)
#                 for e in k_hyperedge_dict:
#                     for n in r_hyperedge_dict:
#                         if r_hyperedge_dict[n] <= k_hyperedge_dict[e]:
#                             MP[ndict[n], edict[e]] = 1
#             if index:
#                 return ndict, edict, MP
#             else:
#                 return MP
#         else:
#             if index:
#                 return {}, {}, np.zeros(1)
#             else:
#                 return np.zeros(1)

#     def _incidence_matrix(
#         self,
#         rank,
#         to_rank,
#         incidence_type: Literal["up", "down"] = "up",
#         weight=None,
#         sparse: bool = True,
#         index: bool = False,
#     ):
#         if rank == to_rank:
#             raise ValueError("incidence must be computed for k!=r, got equal r and k.")
#         if to_rank is None:
#             if incidence_type == "up":
#                 children = self.skeleton(rank)
#                 uidset = self.skeleton(rank + 1)
#             elif incidence_type == "down":
#                 uidset = self.skeleton(rank)
#                 children = self.skeleton(rank - 1)
#             else:
#                 raise TopoNetXError("incidence_type must be 'up' or 'down' ")
#         else:
#             assert (
#                 rank != to_rank
#             )  # incidence is defined between two skeletons of different ranks
#             if (
#                 rank < to_rank
#             ):  # up incidence is defined between two skeletons of different ranks
#                 children = self.skeleton(rank)
#                 uidset = self.skeleton(to_rank)

#             elif (
#                 rank > to_rank
#             ):  # up incidence is defined between two skeletons of different ranks
#                 children = self.skeleton(to_rank)
#                 uidset = self.skeleton(rank)
#         return self._incidence_matrix_helper(children, uidset, sparse, index)

#     def clone(self) -> "CombinatorialComplex":
#         CC = CombinatorialComplex(name=self.name, graph_based=self.graph_based)
#         for cell in self.cells:
#             CC.add_cell(cell, self.cells.get_rank(cell))
#         return CC
#     def coadjacency_matrix(self, rank, via_rank, s: int = 1, index: bool = False):
#         if via_rank is not None:
#             assert rank > via_rank
#         if index:
#             B, row, col = self.incidence_matrix(
#                 via_rank, rank, incidence_type="down", sparse=True, index=index
#             )
#         else:
#             B = self.incidence_matrix(
#                 rank, via_rank, incidence_type="down", sparse=True, index=index
#             )
#         A = incidence_to_adjacency(B)
#         if index:
#             return A, col
#         return A
    
#     @property
#     def dim(self) -> int:
#         return max(self._complex_set.allranks)
    
#     @property
#     def nodes(self):
#         return NodeView(self._complex_set.hyperedge_dict, cell_type=HyperEdge)
#     @property
#     def shape(self) -> tuple[int, ...]:
#         return self._complex_set.shape
#     def skeleton(self, rank: int):
#         """Return skeleton."""
#         return self._complex_set.skeleton(rank)
#     def incidence_matrix(
#         self,
#         rank,
#         to_rank=None,
#         incidence_type: str = "up",
#         weight=None,
#         sparse: bool = True,
#         index: bool = False,
#     ):
#         return self._incidence_matrix(
#             rank, to_rank, incidence_type=incidence_type, sparse=sparse, index=index
#         )
#     def _remove_node(self, node) -> None:
#         self._remove_hyperedge(node)

#     def remove_node(self, node):
#         self._remove_node(node)
#         return self
#     def remove_nodes(self, node_set) -> None:
#         for node in node_set:
#             self.remove_node(node)
#     def add_cell(self, cell, rank=None, **attr):
#         if self.graph_based:
#             if rank == 1:
#                 if not isinstance(cell, Iterable):
#                     TopoNetXError(
#                         "Rank 1 cells in graph-based CombinatorialComplex must be Iterable."
#                     )
#                 if len(cell) != 2:
#                     TopoNetXError(
#                         f"Rank 1 cells in graph-based CombinatorialComplex must have size equalt to 1 got {cell}."
#                     )

#         self._add_hyperedge(cell, rank, **attr)
#         return self
#     def _add_hyperedge_helper(self, hyperedge_, rank, **attr) -> None:
#         if rank in self._complex_set.hyperedge_dict:
#             if hyperedge_ in self._complex_set.hyperedge_dict[rank]:
#                 self._complex_set.hyperedge_dict[rank][hyperedge_].update(attr)
#                 for i in hyperedge_:
#                     if 0 not in self._complex_set.hyperedge_dict:
#                         self._complex_set.hyperedge_dict[0] = {}

#                     if i not in self._complex_set.hyperedge_dict[0]:
#                         self._complex_set.hyperedge_dict[0][frozenset({i})] = {
#                             "weight": 1
#                         }
#             else:
#                 self._complex_set.hyperedge_dict[rank][hyperedge_] = {}
#                 self._complex_set.hyperedge_dict[rank][hyperedge_].update(attr)
#                 for i in hyperedge_:
#                     if 0 not in self._complex_set.hyperedge_dict:
#                         self._complex_set.hyperedge_dict[0] = {}

#                     if i not in self._complex_set.hyperedge_dict[0]:
#                         self._complex_set.hyperedge_dict[0][frozenset({i})] = {
#                             "weight": 1
#                         }
#         else:
#             import pdb;pdb.set_trace()
#             self._complex_set.hyperedge_dict[rank] = {}
#             self._complex_set.hyperedge_dict[rank][hyperedge_] = {}
#             self._complex_set.hyperedge_dict[rank][hyperedge_].update(attr)

#             for i in hyperedge_:
#                 if 0 not in self._complex_set.hyperedge_dict:
#                     self._complex_set.hyperedge_dict[0] = {}
#                 if i not in self._complex_set.hyperedge_dict[0]:
#                     self._complex_set.hyperedge_dict[0][frozenset({i})] = {"weight": 1}

#     def _add_hyperedge(self, hyperedge, rank: int, **attr):
#         if not isinstance(rank, int):
#             raise ValueError(f"rank must be an integer, got {rank}")

#         if rank < 0:
#             raise ValueError(f"rank must be non-negative integer, got {rank}")

#         if isinstance(hyperedge, str):
#             if rank != 0:
#                 raise ValueError(f"rank must be zero for string input, got rank {rank}")
#             else:
#                 if 0 not in self._complex_set.hyperedge_dict:
#                     self._complex_set.hyperedge_dict[0] = {}
#                 self._complex_set.hyperedge_dict[0][frozenset({hyperedge})] = {}
#                 self._complex_set.hyperedge_dict[0][frozenset({hyperedge})].update(attr)
#                 self._aux_complex.add_simplex(Simplex(frozenset({hyperedge}), r=0))
#                 self._complex_set.hyperedge_dict[0][frozenset({hyperedge})][
#                     "weight"
#                 ] = 1
#                 return

#         if isinstance(hyperedge, Hashable) and not isinstance(hyperedge, Iterable):
#             if rank != 0:
#                 raise ValueError(f"rank must be zero for hashables, got rank {rank}")
#             else:
#                 if 0 not in self._complex_set.hyperedge_dict:
#                     self._complex_set.hyperedge_dict[0] = {}
#                 self._complex_set.hyperedge_dict[0][frozenset({hyperedge})] = {}
#                 self._complex_set.hyperedge_dict[0][frozenset({hyperedge})].update(attr)
#                 self._aux_complex.add_simplex(Simplex(frozenset({hyperedge}), r=0))
#                 self._complex_set.hyperedge_dict[0][frozenset({hyperedge})][
#                     "weight"
#                 ] = 1
#                 return
#         if isinstance(hyperedge, Iterable) or isinstance(hyperedge, HyperEdge):
#             if not isinstance(hyperedge, HyperEdge):
#                 hyperedge_ = frozenset(
#                     sorted(hyperedge)
#                 )  # put the simplex in canonical order
#                 if len(hyperedge_) != len(hyperedge):
#                     raise ValueError(
#                         f"a cell cannot contain duplicate nodes,got {hyperedge_}"
#                     )
#             else:
#                 hyperedge_ = hyperedge.elements
#         if isinstance(hyperedge, Iterable) or isinstance(hyperedge, HyperEdge):
#             for i in hyperedge_:
#                 if not isinstance(i, Hashable):
#                     raise ValueError(
#                         f"every element cell must be hashable, input cell is {hyperedge_}"
#                     )
#         if (
#             rank == 0
#             and isinstance(hyperedge, Iterable)
#             and not isinstance(hyperedge, str)
#         ):
#             if len(hyperedge) > 1:
#                 raise ValueError(
#                     "rank must be positive for higher order cells, got rank = 0 "
#                 )

#         self._aux_complex.add_simplex(Simplex(hyperedge_, r=rank))
#         if self._aux_complex.is_maximal(hyperedge_):  # safe to insert the hyperedge
#             # looking down from hyperedge to other hyperedges in the complex
#             # make sure all subsets of hyperedge have lower ranks
#             all_subsets = self._aux_complex.get_boundaries([hyperedge_], min_dim=1)
#             for f in all_subsets:
#                 if frozenset(f) == frozenset(hyperedge_):
#                     continue
#                 if "r" in self._aux_complex[f]:  # f is part of the CC
#                     if self._aux_complex[f]["r"] > rank:
#                         rr = self._aux_complex[f]["r"]
#                         self._aux_complex.remove_maximal_simplex(hyperedge_)
#                         raise ValueError(
#                             "a violation of the combinatorial complex condition:"
#                             + f"the hyperedge {f} in the complex has rank {rr} is larger than {rank}, the rank of the input hyperedge {hyperedge_} "
#                         )

#             self._add_hyperedge_helper(hyperedge_, rank, **attr)
#             if "weight" not in self._complex_set.hyperedge_dict[rank][hyperedge_]:
#                 self._complex_set.hyperedge_dict[rank][hyperedge_]["weight"] = 1
#             if isinstance(hyperedge, HyperEdge):
#                 self._complex_set.hyperedge_dict[rank][hyperedge_].update(
#                     hyperedge._properties
#                 )

#         else:
#             import pdb;pdb.set_trace()
#             all_cofaces = self._aux_complex.get_cofaces(hyperedge_, 0)
#             # looking up from hyperedge to other hyperedges in the complex
#             # make sure all supersets that are in the complex of hyperedge have higher ranks

#             for f in all_cofaces:
#                 if frozenset(f) == frozenset(hyperedge_):
#                     continue
#                 if "r" in self._aux_complex[f]:  # f is part of the CC
#                     if self._aux_complex[f]["r"] < rank:
#                         rr = self._aux_complex[f]["r"]
#                         # all supersets in a CC must have ranks that is larger than or equal to input ranked hyperedge
#                         raise ValueError(
#                             "violation of the combinatorial complex condition : "
#                             + f"the hyperedge {f} in the complex has rank {rr} is smaller than {rank}, the rank of the input hyperedge {hyperedge_} "
#                         )
#             self._aux_complex[hyperedge_]["r"] = rank
#             self._add_hyperedge_helper(hyperedge_, rank, **attr)
#             if "weight" not in self._complex_set.hyperedge_dict[rank][hyperedge_]:
#                 self._complex_set.hyperedge_dict[rank][hyperedge_]["weight"] = 1
#             if isinstance(hyperedge, HyperEdge):
#                 self._complex_set.hyperedge_dict[rank][hyperedge_].update(
#                     hyperedge._properties
#                 )

In [ ]:
from toponetx import CombinatorialComplex as cc
example = cc(name = 'hello')

example.add_cell([1, 2], rank=1)

example.add_cell([1, 3,2], rank=1)

example.add_cell([1, 2, 4, 3], rank=2)

example.add_cell([2, 5], rank=1)

example.add_cell([2, 6, 4], rank=2)
row,column,B2 = example.incidence_matrix(0,1, index=True)
row1,column1,B3 = example.incidence_matrix(0,2,index = True)

In [ ]:
example.incidence_matrix(rank= 1,to_rank = 0,incidence_type='down').shape == (6,3)

In [ ]:
from toponetx import CombinatorialComplex 
def test_incidence_matrix_to_rank_down():
    """Test generating an incidence matrix by setting the down_rank parameter."""
    CC = CombinatorialComplex()
    CC.add_cell([1, 2], rank=1)
    CC.add_cell([1, 3], rank=1)
    CC.add_cell([1, 2, 4, 3], rank=2)
    CC.add_cell([2, 5], rank=1)
    CC.add_cell([2, 6, 4], rank=2)
    B = CC.incidence_matrix(2,0,incidence_type="down")
    assert B.shape == (6, 2)
    assert (
        B.todense()
        == [[1, 0], [1, 1], [1, 0], [1, 1], [0, 0], [0, 1]]
    ).all()

def test_incidence_matrix_to_rank_down_without_rank():
    """Test generating an incidence matrix by setting the down_rank parameter without mentioning rank."""
    CC = CombinatorialComplex()
    CC.add_cell([1, 2], rank=1)
    CC.add_cell([1, 3], rank=1)
    CC.add_cell([1, 2, 4, 3], rank=2)
    CC.add_cell([2, 5], rank=1)
    CC.add_cell([2, 6, 4], rank=2)
    B = CC.incidence_matrix(2,incidence_type="down")
    assert B.shape == (3, 2)
    assert (
        B.todense()
        == [[1, 0], [1,0], [0, 0]]
    ).all()

def test_incidence_matrix_to_rank_with_wrong_incidence_type():
    '''Test generating an incidence matrix by mentioning wrong rank.'''
    CC = CombinatorialComplex()
    CC.add_cell([1, 2], rank=1)
    CC.add_cell([1, 3], rank=1)
    CC.add_cell([1, 2, 4, 3], rank=2)
    CC.add_cell([2, 5], rank=1)
    CC.add_cell([2, 6, 4], rank=2)
    with pytest.raises(TopoNetXError):
        CC.incidence_matrix(2,incidence_type="wrong")

def test_incidence_matrix_with_equal_rank():
    '''Test generating an incidence matrix by having equal rank.'''
    CC = CombinatorialComplex()
    CC.add_cell([1, 2], rank=1)
    CC.add_cell([1, 3], rank=1)
    CC.add_cell([1, 2, 4, 3], rank=2)
    CC.add_cell([2, 5], rank=1)
    CC.add_cell([2, 6, 4], rank=2)
    with pytest.raises(ValueError):
        CC.incidence_matrix(1,1)

def test_incidence_dict():
    '''Test generating an incidence dictionary.'''
    CC = CombinatorialComplex()
    CC.add_cell([1, 2], rank=1)
    CC.add_cell([1, 3], rank=1)
    CC.add_cell([1, 2, 4, 3], rank=2)
    CC.add_cell([2, 5], rank=1)
    CC.add_cell([2, 6, 4], rank=2)
    assert CC.incidence_dict == {1: {frozenset({1, 2}): {'weight': 1},
  frozenset({1, 3}): {'weight': 1},
  frozenset({2, 5}): {'weight': 1}},
 0: {frozenset({1}): {'weight': 1},
  frozenset({2}): {'weight': 1},
  frozenset({3}): {'weight': 1},
  frozenset({4}): {'weight': 1},
  frozenset({5}): {'weight': 1},
  frozenset({6}): {'weight': 1}},
 2: {frozenset({1, 2, 3, 4}): {'weight': 1},
  frozenset({2, 4, 6}): {'weight': 1}}}

def test_dim():
    '''Test for the dimensionality of the CombinatorialComplex object.
    Gets the highest rank of the cells in the CombinatorialComplex object.'''
    CC = CombinatorialComplex()
    CC.add_cell([1, 2], rank=1)
    CC.add_cell([1, 3], rank=1)
    CC.add_cell([1, 2, 4, 3], rank=2)
    CC.add_cell([2, 5], rank=1)
    CC.add_cell([2, 6, 4], rank=3)
    assert CC.dim == 3

def test_repr():
    '''Test the represntation function of the CombinatorialComplex object by mentioning a name.'''
    CC = CombinatorialComplex(name = 'sampleobject')
    assert repr(CC) == "CombinatorialComplex(name='sampleobject')"

def test_contains():
    '''Test whether the contains method works correctly.'''
    CC = CombinatorialComplex()
    CC.add_cell([1, 2], rank=1)
    CC.add_cell([1, 3], rank=1)
    CC.add_cell([1, 2, 4, 3], rank=2)
    CC.add_cell([2, 5], rank=1)
    CC.add_cell([2, 6, 4], rank=2)
    assert [(1)] in CC.nodes
    assert [2] in CC.nodes
    assert [3] in CC.nodes
    assert [4] in CC.nodes
    assert [5] in CC.nodes
    assert [6] in CC.nodes

def test_set_item():
    CC = CombinatorialComplex()
    CC.add_cell([1],rank = 3)
    CC.add_cell([1,2],rank = 3)
    ## Updating an node attribute present in the CombinatorialComplex object.
    CC.__setitem__([1],{'weights':1})
    ## Setting a cell attribute present in the CombinatorialComplex object.
    CC.__setitem__([1,2],{'weights':1})
    assert CC._complex_set.hyperedge_dict[3][frozenset([1,2])] == {'weights':1}
    assert CC.nodes[1]['weights'] == 1

def test_degree():
    '''Test for the degree function.'''
    CC = CombinatorialComplex()
    CC.add_cell([1, 2], rank=1)
    CC.add_cell([1, 3], rank=1)
    CC.add_cell([1, 2, 4, 3], rank=2)
    CC.add_cell([2, 5], rank=1)
    CC.add_cell([2, 6, 4], rank=2)
    CC.add_cell([1, 2], rank=2)
    assert CC.degree(1) == 2
    with pytest.raises(TopoNetXError):
        assert CC.degree(1,-1) == TopoNetXError("Rank must be positive")
    assert CC.degree(2,2) == 3
    with pytest.raises(KeyError):
        node = 7
        assert CC.degree(node,2) == KeyError(f"Node {node} not in Combinatorial Complex.")

def test_size():
    '''Test for the size function.'''
    CC = CombinatorialComplex()
    CC.add_cell([1, 2], rank=1)
    CC.add_cell([1, 3], rank=1)
    CC.add_cell([1, 2, 4, 3], rank=2)
    CC.add_cell([2, 5], rank=1)
    CC.add_cell([2, 6, 4], rank=2)
    CC.add_cell([1, 2], rank=2)
    assert CC.size(1) == 1
    with pytest.raises(TopoNetXError):
        CC.size(frozenset([1,2,3])) == TopoNetXError("Input cell is not in cells of the CC")

def test_num_nodes_and_cells():
    '''Test for number of nodes and number of cells'''
    CC = CombinatorialComplex()
    CC.add_cell([1, 2], rank=1)
    CC.add_cell([1, 3], rank=1)
    CC.add_cell([1, 2, 4, 3], rank=2)
    CC.add_cell([2, 5], rank=1)
    CC.add_cell([2, 6, 4], rank=2)
    CC.add_cell([1, 2], rank=2)
    y1 = HyperEdge([1,2],rank = 1)
    y2 = HyperEdge([1,3],rank = 1)
    assert CC.number_of_nodes() == 6    
    assert CC.number_of_nodes([1,2]) == 2
    assert CC.number_of_cells() == 12
    assert CC.number_of_cells([y1,y2]) == 2

def test_order():
    '''Test for the order function.'''
    CC = CombinatorialComplex()
    CC.add_cell([1, 2,3,4,5,6,7,8], rank=1)
    assert CC.order() == 8

def test_remove_node():
    '''Test if the node is removed correctly from the CombinatorialComplex object.'''
    CC = CombinatorialComplex()
    CC.add_cell([1,2,3,4,5,6,7,8], rank= 1)
    
